In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# divide data into train and test sets
from sklearn.model_selection import train_test_split 

In [ ]:
# load dataset
test_df = pd.read_csv('test.csv')

In [ ]:
# LOOK AT THE DATA AND SEE IF THERE ARE ANY SUMMARY STATISTICS THAT MIGHT GIVE YOU SOME INSIGHTS

In [ ]:
# print shape of train and test dataframes
print(train_df.shape) 
print(test_df.shape) 

In [ ]:
# print first 5 rows of train dataframe
print(train_df.head())

In [ ]:
# print train dataframe info 
print(train_df.info())

In [ ]:
# select all columns that are objects
categorical_columns = train_df.select_dtypes(include=['object'])
# print number of categorical columns
print(f'There are {len(categorical_columns.columns.tolist())} categorical columns in the dataset:')
# for each column in categorical columns, print column name and number of unique values
for cols in categorical_columns.columns: 
    print(cols,':', len(categorical_columns[cols].unique()),'labels')

In [ ]:
# DATA CLEANING AND FEATURE ENGINEERING

In [ ]:
# create booleans for each of the embarkment points
train_df['Embarked'] = train_df['Embarked'].fillna('S')
train_df['Embarked'] = train_df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
# print the boolean variable for each of the embarkment points
print(train_df['Embarked'].value_counts())

In [ ]:
# Create a boolean for is male. 
#not sure if this is how you do it

train_df_sex = pd.get_dummies(train_df['Sex'])
isMale = train_df_sex['male']
isMale

In [ ]:
#Create a boolean for whether someone has a cabin
PeopleWithCabin = train_df[train_df['Cabin'].isnull() == False]
PeopleWithoutCabin = train_df[train_df['Cabin'].isnull()]

#still working on how to create the boolean for this 

In [ ]:
# check for missing values in the data
print(train_df.isnull().sum())

In [ ]:
# create boolean variable for each of the embarkment points
# for each value in the Embarked column
for uwu in train_df.Embarked.unique(): 
    # if it is a string (just to be safe lol)
    if type(uwu) == str: 
        # create a new column with the boolean value
        train_df['emb' + uwu] = (train_df.Embarked == uwu) * 1 

In [ ]:
# create boolean variable for is male
train_df['is_male'] = (train_df.Sex == 'male') * 1 

In [ ]:
# create boolean variable for has cabin
train_df.loc[:, 'has_cabin'] = 0
train_df.loc[train_df.Cabin.isna(), 'has_cabin'] = 1

In [ ]:
# fill missing age values as 100 
train_df['Age'].fillna(100)

In [ ]:
# divide the data into a training set and a testing set
X = train_df[["Pclass", "Age", "Sex", "SibSp", "Parch", "Embarked"]]
Y = train_df["Survived"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0)